缓慢降温没用，增加探索性没用，探索性破破奖励信号。

375100：农民出牌，手牌对J，2。地主最大为对A，场上无大小王。农民出2、对J即胜，实际出单张J。农民胜。

In [23]:
import numpy as np
import keras as k
import rules
import arena as arn
import bot_base2
import bot_sarsa
ARENA = arn.ARENA
bot_rd = arn.bot_rd
bot_me = arn.bot_me


In [42]:
# base2 188150 << 263300 < 386150 >> 388500 
# model = k.models.load_model("model_base_allrecords/m{}.keras".format(17544))
# model = k.models.load_model("model_base2/m{}.keras".format(188150))

def bot1_f(verbos=0):
    model = k.models.load_model("model_sarsa/m{}.keras".format(9550))
    return bot_sarsa.BOT(model, verbos=verbos)

def bot2_f(verbos=0):
    model = k.models.load_model("model_base2/m{}.keras".format(9550))
    return bot_base2.BOT(model, verbos=verbos)
    # model = k.models.load_model("model_sarsa/m{}.keras".format(6900))
    # return bot_sarsa.BOT(model, verbos=verbos)

def bot_rd_f():
    return bot_rd

In [43]:
def ft(N, bot1_f, bot2_f):
    s1, s2 = 0, 0
    for i in range(N):
        cards = rules.CARDS.copy()
        np.random.shuffle(cards)
        
        arena = ARENA(verbos=0, cards=cards.copy())
        arena.registerbot([bot1_f(), bot2_f(), bot2_f()])
        arena.wholegame()
        s1 += (arena.winner == 0)
        
        arena2 = ARENA(verbos=0, cards=cards.copy())
        arena2.registerbot([bot2_f(), bot1_f(), bot1_f()])
        arena2.wholegame()
        s2 += (arena2.winner == 0)
        
    # for i in range(N):
    #     arena = ARENA(verbos=0)
    #     arena.registerbot([BOT(model_0), BOT(model), BOT(model)])
    #     # arena.registerbot([BOT(model_0), bot_rd, bot_rd])
    #     arena.wholegame()
    #     s2 += (arena.winner == 0)
    # return s1
    return s1, s2

In [ ]:
ft(100, bot1_f, bot2_f)

In [37]:
arena = arn.ARENA(1, True)
arena.registerbot([bot1_f(1), bot1_f(1), bot1_f(1)])
arena.wholegame()

-0.014724673 5 5 6 6 7 7 8 8
-0.24173559 6 6 7 7 8 8
-0.24607272 5 5 6 6 7 7
-0.27073085 3 4 5 6 7 8 9 10 J Q
-0.33372837 3 4 5 6 7 8 9 10 J Q K
 0 :  地主 : 5 5 6 6 7 7 8 8 | 3 4 9 10 10 J J Q K K 2 小王
-0.061587706 
 1 : 农民1 :  | 3 3 4 6 7 8 9 9 10 10 Q K A 2 2 2 大王
-0.11077915 
 2 : 农民2 :  | 3 4 4 5 5 6 7 8 9 J J Q Q K A A A
-0.07624193 4
-0.10187233 J J
-0.11287355 3
-0.12792358 K K
-0.1409927 9 10 J Q K
 3 :  地主 : 4 | 3 9 10 10 J J Q K K 2 小王
-0.041451156 大王
-0.08602911 A
-0.09123943 2
-0.1006922 K
-0.11831808 
 4 : 农民1 : 大王 | 3 3 4 6 7 8 9 9 10 10 Q K A 2 2 2
0.008405651 
 5 : 农民2 :  | 3 4 4 5 5 6 7 8 9 J J Q Q K A A A
-0.06907293 
 6 :  地主 :  | 3 9 10 10 J J Q K K 2 小王
0.010763095 6 7 8 9 10
-0.012393047 A 2 2 2
-0.03118168 3 3 2 2 2
-0.034432914 K 2 2 2
-0.045873202 2 2
 7 : 农民1 : 6 7 8 9 10 | 3 3 4 9 10 Q K A 2 2 2
0.02053519 
 8 : 农民2 :  | 3 4 4 5 5 6 7 8 9 J J Q Q K A A A
0.19785784 9 10 J Q K
-0.0798868 
 9 :  地主 : 9 10 J Q K | 3 10 J K 2 小王
-0.21841455 
10 : 农民1 :  | 3 3 4 9 

In [8]:
def bot_mc_f(basebot, verbos=2, RECORD=True, MAXWIDTH=3, NSIMS_FACTOR=50, NSIMS_MAX=1000):
    return mc.BOT(basebot=basebot, verbos=verbos, RECORD=RECORD, MAXWIDTH=MAXWIDTH, NSIMS_FACTOR=NSIMS_FACTOR, NSIMS_MAX=NSIMS_MAX)

In [6]:
import re

def repl(matched):
    d = {"11": "J", "12": "Q", "13": "K", "14": "A", "15": "2", "16": "小王", "17": "大王"}
    return d[matched[0]]

def arenabyscratch(l):
    cards = [rules.str2vec(re.sub(r'1[1-7]', repl, v)) for v in re.split(",", l)]
        
    arena=object.__new__(arn.ARENA)
    arena.verbos = 1
    arena.init = np.array(cards, int)
    arena.remain = arena.init.copy()
    arena.lastplay = np.zeros((3, 15), int)
    arena.pos = 0
    arena.b1 = 2
    arena.b2 = 1
    arena.round = 0
    arena.bot = []
    arena.gameover = False
    arena.winner = None
    arena.getdata()
    arena.RECORD = True
    arena.records = []
    if arena.RECORD:
        arena.records.append(arena.copy())
    return arena

arena = arenabyscratch("3 4 4 4 4 5 5 7 8 9 9 9 10 11 12 13 13 14 14 17, 3 5 5 6 6 6 7 7 7 8 9 10 11 12 13 13 15, 3 3 6 8 8 10 10 11 11 12 12 14 14 15 15 15 16")

In [31]:
a1 = arena.records[0].copy(1, True)
a1.registerbot([bot1_f(1), bot1_f(1), bot1_f(1)])
# a1.wholegame()

In [27]:
a1 = arena.records[0].copy(1, True)
bot = bot_conv.BOT(bot_conv.BOT.createmodel())
a1.registerbot([bot, bot, bot])

In [28]:
a1.bot

In [30]:
bot.showChoices(a1)

AttributeError: 'BOT' object has no attribute 'arena'

In [15]:
a1 = arena.records[0].copy(0, True)
a1.update(a1.getChoices()[0])

In [ ]:
model1 = k.models.load_model("model_base2/m{}.keras".format(391350))
model2 = k.models.load_model("model_base2/m{}.keras".format(391300))

In [23]:
w1 = model1.weights[1].numpy().reshape(-1)
w2 = model2.weights[1].numpy().reshape(-1)

In [24]:
w1[0:10], w2[0:10]

(array([-0.00342188, -0.00154246, -0.0020585 , -0.00466309, -0.00450408,
         0.06086061,  0.00401902, -0.00338081,  0.00091003,  0.01286665],
       dtype=float32),
 array([-0.0034434 , -0.00159783, -0.00205496, -0.00465686, -0.00447862,
         0.06076964,  0.0040254 , -0.00332928,  0.00091905,  0.01286706],
       dtype=float32))

In [31]:
f = open("base2.txt", "a")
f.write(" ".join(["391400", "sdlkfj"])+"\n")
f.close()